In [1]:
%pylab inline
import os, sys
sys.path.append("../")
import traml
import db
from scipy.optimize import minimize
import copy
import time

Populating the interactive namespace from numpy and matplotlib


In [6]:
PAZO = 500603
PMMA = 500201
sio2 = 111401
mgf2 = 105404
a = traml.Stack()
a.library.search('Treharne')

|     ID | SHELF   | BOOK   | PAGE     | DESCRIPTION                    |
|--------+---------+--------+----------+--------------------------------|
| 114203 | main    | CdS    | Treharne | Treharne et al. 2011: Thin fil |
| 116605 | main    | CdTe   | Treharne | Treharne et al. 2011: Thin fil |
| 401401 | other   | Al:ZnO | Treharne | Treharne et al. 2011: Thin fil |


In [3]:
def fun(x, stack, layers, c, delta, TR):
    for i in layers:
        stack.d(i, x[i-1])

    for p in x:
        p = np.sqrt(x[0]**2)
    x[0]=np.sqrt(x[0]**2)
    #min = 1-stack.average(TR, c, delta, o='a')
    min = stack.average(TR, c, delta)
   
    return min**10

In [4]:
def minTR(stack, film_range, c, delta, TR='T'):
    
    x = []
    layers = []
    bnds = []
    for i in range(film_range[0],film_range[1]+1):
        x.append(stack.config[i][1])
        layers.append(i)
        bnds.append(stack.config[i][3])
        
    result = minimize(fun, x, args=(stack, layers, c,delta, TR), bounds = bnds,tol=1e-30, method='SLSQP')
    
    
    return result

In [5]:
c = traml.Stack()
c.set_range([0.4, 0.8])

peak = 0.65
delta = 0.01
stack_list = []
R=[]
n=[]

N = 40
timer=[]

for i in range(0, N):
    start = time.clock()
    #if i % 2 == 0:
       # print 'even', i
        #c.add(PMMA,100, bnds=(50,250))
    # else:
        #c.add(PAZO,100, bnds=(50,250))
    
    c.add(PMMA, 100, bnds=(0,250))
    c.add(PAZO, 100, bnds=(0,250))
    
    
    result = minTR(c, [1,len(c.config)-1], peak, delta, TR='T')
    R.append(c.average('R',peak,delta))
    n.append(i+1)
    
    stack_list.append(copy.deepcopy(c))
  
    print '============================================================================================'
   
    print i+1, ': ', 'R = ', c.average('R',peak,delta)
    end = time.clock()
    duration = end-start
    timer.append(duration)
 
    

TypeError: 'NoneType' object has no attribute '__getitem__'

In [ ]:
s = []
for item in timer:
    s.append(2*pi*(item)**2)

In [ ]:
fig = figure(figsize=(9,7))
ax = fig.add_subplot(111)
R = []

for stack in stack_list:
    R.append(stack.average('R',0.65,0.01))
    
ax.scatter(n, R, s=s, alpha = 0.1)
ax.set_xlim([1,25])
ax.set_ylim([0,1.1])
ax.plot(n, R, 'o-', linewidth=2, color='red', markersize = 2)




In [ ]:
len(n)

In [ ]:
c.set_range([0.4, 1.0])
c.plot()